In [24]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import math
import random
from ukr_stemmer3 import UkrainianStemmer

In [25]:
data_set = pd.read_csv('./collectdata/comments_ua.txt', sep = ':::::', names = ["score", "comment"])



/home/ksenia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data_set.groupby("score").count()

,comment
score,
1,70
2,84
3,171
4,605
5,2196


In [4]:
msk = np.random.rand(len(data_set)) < 0.8
train = data_set[msk]
test = data_set[~msk]

In [28]:
def to_base_form(use_lemma, word):
    if (use_lemma):
        return UkrainianStemmer(word.lower()).stem_word()
    return word.lower()

def train_model(stop_words=[], use_lemma=False, **kwargs):
    
    def calc_probabilities_in_class(data):
        bag_of_words = {}
        for index, row in data.iterrows():
            words = nltk.word_tokenize(row['comment'])
            for word in words:
                base_form = to_base_form(use_lemma, word)
                if not base_form in stop_words:
                    if not base_form in bag_of_words:
                        bag_of_words[base_form] = 1
                    else:
                        bag_of_words[base_form] = bag_of_words[base_form] + 1
        return bag_of_words
    
    probabilities = {}
    for key in kwargs:
        probabilities[key] = calc_probabilities_in_class(kwargs[key])
    return probabilities


def predict_model_comment(model, comment, stop_words = [], use_lemma = False):
    
    def comment_log_probabitlity(words, class_model, total_words, total_unique_words):
        denominator = sum(class_model.values()) + total_unique_words
        log_prob = math.log(sum(class_model.values()) / total_words)
        for word in words:
            n = class_model[word] if word in class_model else 0 
            log_prob += math.log((n + 1)/denominator)
        return log_prob
        
    log_probablitites = {}
    words = [to_base_form(use_lemma, word) for word in nltk.word_tokenize(comment) if word.lower not in stop_words]
    
    total_words = 0
    total_unique_words = 0
    for key in model:
        total_words += sum(model[key].values())
        total_unique_words += len(model[key])
        
    for key in model:
        log_probablitites[key] = comment_log_probabitlity(words, model[key], total_words, total_unique_words)
    return log_probablitites

def print_report(true_positive, true_negative, false_positive, false_negative):
    print("True Positive:", true_positive, "; True negative:", 
          true_negative, "; False positive:", false_positive, "; False negative:", false_negative)
    print("Presision:", true_positive/(true_positive + false_positive),
          "; Recall:", true_positive/(true_positive + false_negative), "\n")

def predict_on_set(predict_comment_lambda, test_set, labeled_negative):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0

    for index, row in test_set.iterrows():
        max_prob = float('-inf')
        max_key = ""
        probs = predict_comment_lambda(row["comment"])
        for key_class in probs:
            if probs[key_class] > max_prob:
                max_prob = probs[key_class]
                max_key = key_class
        is_negative_eval = max_key == 'negative'
        is_negative_label = labeled_negative(row["score"]) 
        #print("is_negative_eval:", is_negative_eval, " is_neg_label:", is_negative_label)
      
        if is_negative_eval and is_negative_label:
            true_negative += 1
            
        if not is_negative_eval and not is_negative_label:
            true_positive += 1
            
        if not is_negative_eval and is_negative_label:
            false_positive += 1
            
        if is_negative_eval and not is_negative_label:
            false_negative += 1
        
    return print_report(true_positive, true_negative, false_positive, false_negative)
        
def predict_random_comment(train_data, comment):
    total_positive = train_data[train_data["score"] > 3]["comment"].count()
    positive_prob = total_positive / train_data["comment"].count()

    if random.random() > positive_prob:
        return {"positive": 0, "negative": 1}
    return {"positive": 1, "negative": 0}

Check pure model (no rules or sentiment dictionary) and compare it with random

In [29]:
model = train_model(positive = train[train["score"] > 3], negative = train[train["score"] <= 3])
print("For pure model on test(just to check that algorythm is corect):")
predict_on_set(lambda comment: predict_model_comment(model, comment), train, lambda x: x <= 3)
print("For pure model on train:")
predict_on_set(lambda comment: predict_model_comment(model, comment), test, lambda x: x <= 3)
print("For random:")
predict_on_set(lambda comment: predict_random_comment(train, comment), test, lambda x: x <= 3)

For pure model on test(just to check that algorythm is corect):
True Positive: 2216 ; True negative: 108 ; False positive: 147 ; False negative: 2
Presision: 0.9377909437156158 ; Recall: 0.9990982867448152 

For pure model on train:
True Positive: 563 ; True negative: 3 ; False positive: 67 ; False negative: 20
Presision: 0.8936507936507937 ; Recall: 0.9656946826758147 

For random:
True Positive: 532 ; True negative: 6 ; False positive: 64 ; False negative: 51
Presision: 0.8926174496644296 ; Recall: 0.9125214408233276 



In [35]:
# check classifier from nltk
all_words = set(word.lower() for index,row in train.iterrows() for word in nltk.word_tokenize(row["comment"]))
train_as_words = [({word: (word in nltk.word_tokenize(row["comment"])) for word in all_words},
                   "negative" if row["score"] <= 3 else "positive")  for index,row in train.iterrows()]

classifier = nltk.NaiveBayesClassifier.train(train_as_words)
classifier.show_most_informative_features()
    
def prdic_by_nltk(comment):
    global all_words
    global classifier
    test_sent_features = {word.lower(): (word in nltk.word_tokenize(comment.lower())) for word in all_words}
    label = classifier.classify(test_sent_features)
    return {"positive": (1 if label == "positive" else 0), "negative": (1 if label == "negative" else 0)}

predict_on_set(prdic_by_nltk, test, lambda x: x <= 3)

[({'ніхто': False, 'есть': False, 'скоротився': False, 'microsoft': False, 'мега': False, 'постаравшись': False, 'band2': False, 'зображенням': False, '85': False, 'энергии': False, 'собі': False, 'надути': False, 'краплі': False, 'позитивного': False, 'палиці': False, 'матовий': False, 'карман': False, 'беговел': False, 'пообіцяли': False, 'працювало': False, 'страшний': False, 'аналізи': False, 'натошак': False, 'якісно': False, 'аккума': False, 'добрий': False, 'раціон': False, 'ситечко': False, '15,6': False, '24': False, 'символічну': False, 'маківці': False, 'перевдягати': False, 'zenfone': False, 'насторожено': False, 'хвилинки': False, 'навчилась': False, 'матеріалів': False, 'акамулятором': False, 'різноманітні': False, 'покупка': False, 'тренажер': False, 'коментар': False, 'передачею': False, 'весь': False, 'простіше': False, 'потратив': False, 'подарила': False, 'побував': False, '6.200': False, 'blaster': False, 'використовує': False, 'соці': False, 'потроху': False, 'прош

Most Informative Features
                 дешевий = True           negati : positi =     26.0 : 1.0
                  жодної = True           negati : positi =     20.2 : 1.0
                  певний = True           negati : positi =     20.2 : 1.0
               поставила = True           negati : positi =     20.2 : 1.0
                  реагує = True           negati : positi =     15.6 : 1.0
                 ремінця = True           negati : positi =     14.4 : 1.0
               батарейку = True           negati : positi =     14.4 : 1.0
                зроблена = True           negati : positi =     14.4 : 1.0
                   пробу = True           negati : positi =     14.4 : 1.0
                   точні = True           negati : positi =     14.4 : 1.0
True Positive: 569 ; True negative: 14 ; False positive: 56 ; False negative: 14
Presision: 0.9104 ; Recall: 0.9759862778730704 



In [34]:
tonal_dict = pd.read_csv('../../../../sources/tone-dict-uk.tsv', sep = '\t', names = ["word", "sentiment"])

def predict_comment_with_tonal(model, comment, skip_contradict = False):
    
    def sentiment_score():
        positive = 0
        negative = 0
        for index, row in tonal_dict.iterrows():
            if row["word"] in comment:
                if int(row["sentiment"]) > 0:
                    positive += int(row["sentiment"])
                else:
                    negative += int(row["sentiment"])
        #use only if there is no contradictions
        if skip_contradict and positive != 0 and negative != 0:
            return 0
        return positive if positive > -negative else negative
            
    score = sentiment_score()
   
    if score > 0:
        return {"positive": 1, "negative":0}
    if score < 0:
        return {"negative": 0, "positive":1}
    
    return predict_model_comment(model, comment)

print("Test for use sentiment dictionary (if both + and - select lareger module)")  
predict_on_set(lambda comment: predict_comment_with_tonal(model, comment), test, lambda x: x <= 3)

print("Test for use sentiment dictionary (if both + and - skip)")  
predict_on_set(lambda comment: predict_comment_with_tonal(model, comment, True), test, lambda x: x <= 3)


Test for use sentiment dictionary (if both + and - select lareger module)
True Positive: 305 ; True negative: 31 ; False positive: 39 ; False negative: 278
Presision: 0.8866279069767442 ; Recall: 0.5231560891938251 

Test for use sentiment dictionary (if both + and - skip)
True Positive: 249 ; True negative: 44 ; False positive: 26 ; False negative: 334
Presision: 0.9054545454545454 ; Recall: 0.42710120068610635 



In [32]:
stop_words = ['і', 'на', '.', ',', 'в', '(', ')', 'а', 'за', 'у', 'б']
model = train_model(stop_words, True, positive = train[train["score"] > 3], negative = train[train["score"] <= 3])
print("For train with stop words and stemming:")
predict_on_set(lambda comment: predict_model_comment(model, comment, stop_words, True), test, lambda x: x <= 3)


For train with stop words and stemming:
True Positive: 461 ; True negative: 34 ; False positive: 36 ; False negative: 122
Presision: 0.9275653923541247 ; Recall: 0.79073756432247 



In [33]:
for word in model["positive"]:
    print(word)

мен
сподоб
!
важк
ал
це
лиш
перш
тиждень.головн
результат.крутит
потрібн
не
менш
30
хв
пар
рок
купил
розетц
хул
хуп
1,1
кг
пройш
час
вирішил
що
вже
можн
перейт
більш
серйозн
ваг
так
агрегат
замовил
2,8
як
важч
небул
круг
задоволен
синяк
нем
бо
круч
постійн
бок
оч
тают
дуж
дяк
швидк
доставк
рекоменд
вагом
плюс
цьог
обруч
йог
вага.тіл
отрим
навантаженн
коротк
проміжок
ніж
з
аналогічн
при
цьом
залиш
синц
тіл
я
залишил
задоволенн
нов
придбанн
тижден
тільк
про
результ
ще
ран
говорит
крутит
подобаєт
все
зручн
без
обійшл
хоч
одяг
під
футболк
пояс
допомогл
раз
хвилин
кожн
днем
збільшув
зан
прийш
пошкоджен
акуратн
запакован
замовлял
2
том
жодн
пошкодувал
допомаг
підтримув
себ
форм
легк
збираєт
розбираєт
обійт
тимчас
проблем
брал
подарунок
подруг
класн
штук
займ
набагат
зручніш
звичайн
залізн
березн
2017
складаєт
треб
звикнут
та
правильн
використовув
щоб
бул
добр
ден
замов
для
дружин
користуєт
дні
гарн
якіст
сво
цін
...
головн
регулярн
ним
користув
ваш
фігур
набул
елегантн
радж
ус
хорош
міцн
дос

вімінн
вокаліст
стоик
реченн
навіщ
шкір
нікол
порвет
зекономит
ремонт
каподастр
5.2
міліметр
кап
влаштув
ікщ
електро-гітар
искал
відносн
мід
клавіатур
миди-клавиатур
зачетн
випир
лакуванн
гра
коляр
сб
дороговизн
бомбезн
репетитор
f310чи
ibanez
pf15
новачок
зібран
навчит
задоволені.чудов
новичк
сповн
дифект
спостерігаєт
гитар
жилізн
гарніш
ямах
см40
с40
глух
виготовлен
електронік
fender
squier
stratocaster
мексиканськ
принаймн
експерт
грал
backup
камерн
гітарк
симпатичн
тихеньк
варіант.рекоменд
фон
забер
2011
процесор
digitech
rp
155
звучит
д.адар
ехп
120
9-42
недивл
тнкі
агресивн
задовольня
підключ
студі
оборудуванн
рах
дужеякісн
близеньк
укулел
пришл
м.бердич
оператор
спілкувал
ввічлив
перепитувал
прикр
своєрідн
не-деревян
забарвленн
кілочк
притиск
демократичн
негучн
бряжч
опустит
кобилочк
підструнник
виймаєт
погодостійкіст
надаєт
підлітк
полікарбонатн
зїден
он
детальн
//bandurka.etnoua.info/novyny/bandurka-z-ukulele-koncert-korala-puc-20/
звичайнісіньк
синтезатор
автоакомпанемент
чим